In [5]:
from pymongo import MongoClient
import os, re
from sklearn import linear_model
import numpy as np

MONGO_USER = os.environ['MONGO_USER']
MONGO_PASS = os.environ['MONGO_PASS']
MONGO_URL = os.environ['MONGO_URL']

### Connecting to the database

In [6]:
uri = "mongodb://%s:%s@%s" % (
    MONGO_USER, MONGO_PASS, MONGO_URL)

client = MongoClient(uri)
db = client['HonoluluProperty']

### Finding the houses sold in 2017

In [7]:
cursor = db['hnl_county_data'].find({'SalesInformation.0.saleDate': { '$regex': '.*201[0-8]' }})

houses2017 = list(cursor)

In [42]:
def saleInfoExtraction(house):
    for sale in house['SalesInformation']:
        if sale['saleAmount']:
            return sale['saleDate'].strip()[-4:], sale['saleAmount']
        
    return np.nan, np.nan

def assessmentInfo(house, saleYear):
    for info in house['assessmentInformation']:
        if int(info['assessmentYear'].strip()[-4:]) < int(saleYear[1:]):
            return info['totalPropertyAssessedValue']
        
    return np.nan

def featureExtraction(house):
    print('HA')
    features = []
    tmk = house['tmk']
    propertyClass = house['OwnerAndParcelInformation']['propertyClass']
    landArea = house['OwnerAndParcelInformation']['landArea(ApproximateSqFt)']
        # assessmentInfo(house, saleInfoExtraction(house)[0]),
    numOfAssesments = len(house['assessmentInformation'])
    footage = house['landInformation'][0]['squareFootage']
    rebuilds = len(house['landInformation'])
    occupancy = house['residentialImprovementInformation'][0]['occupancy'] if house['residentialImprovementInformation'] else 0
    houseArea = house['residentialImprovementInformation'][0]['squareFeet'] if house['residentialImprovementInformation'] else 0
    yearBuilt = house['residentialImprovementInformation'][0]['yearBuilt'] if house['residentialImprovementInformation'] else 0
    bedrooms = house['residentialImprovementInformation'][0]['bedrooms'] if house['residentialImprovementInformation'] else 0
    baths = house['residentialImprovementInformation'][0]['fullBaths'] if house['residentialImprovementInformation'] else 0
    halfBaths = house['residentialImprovementInformation'][0]['halfBaths'] if house['residentialImprovementInformation'] else 0
        # len(house['PermitInformation'] if house['PermitInformation'] else 0),
    numSales = len(house['SalesInformation'])
    
    return features

In [43]:
labels = [saleInfoExtraction(house) for house in houses2017]
features = [featureExtraction(house) for house in houses2017]
print(labels[:10], features[:10])

HA
HA
HA
HA
HA
HA
HA
HA
HA
HA


KeyError: 'residentialImprovementInformation'

In [41]:
houses2017[10]

{'OwnerAndParcelInformation': {'OwnerName': 'K PROPERTIES INC  Fee Owner',
  'ParcelNumber': '260180190000',
  'dataCurrentAsOf': 'February 5, 2018',
  'landArea(Acres)': '0.058',
  'landArea(ApproximateSqFt)': '2,526',
  'legalInformationLOT2072201SFLOT208325SFMAP22LCAPP551TOG/R-O-W': '',
  'locationAddress': '2176 KALAKAUA AVE',
  'parcelMap': '',
  'platMap': '',
  'projectName': 'PRADA',
  'propertyClass': 'COMMERCIAL'},
 'PermitInformation': [{'date': ' 01/14/2015 ',
   'permitAmount': ' $ 700,000 ',
   'permitNumber': '759851',
   'reason': 'ALTERATION '},
  {'date': ' 05/02/2006 ',
   'permitAmount': ' $ 1,628,320 ',
   'permitNumber': '595884',
   'reason': 'ALTERATION '},
  {'date': ' 05/02/2006 ',
   'permitAmount': ' $ 40,000 ',
   'permitNumber': '595873',
   'reason': 'ALTERATION '},
  {'date': ' 04/28/2000 ',
   'permitAmount': ' $ 43,500 ',
   'permitNumber': '508005',
   'reason': ' '},
  {'date': ' 09/05/1997 ',
   'permitAmount': ' $ 2,500 ',
   'permitNumber': '40773

In [ ]:
client.close()